### Initial imports

In [37]:
import io
import sys

### Eenvironment

In [ ]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [32]:
from openai import OpenAI
import anthropic

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-sonnet-4-20250514"

In [4]:
system_message = "You are an assistant that helps to rename poorly defined variables function titles in a meaningful way (only if they are poorly defined), add function description and comments to the existing python code without changing implementation methods themselves. "
system_message += "If comments for a certain function/class are already present - don't be redundant, don't repeat the same. "
system_message += "However you can rewrite the existing comment if it lacks comleteness or if its meaning can be expressed clearer and easier to understand. "

In [15]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [16]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [5]:
def user_prompt_for(python):
    user_prompt = "Refactor this Python code in C++ with implementation that produces identical output. "
    user_prompt += "Respond only with python code, so I could copypaste your whole response to the python file and execute.\n"
    user_prompt += python
    return user_prompt

In [6]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [33]:
def stream_gpt(python):    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [34]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n','').replace('```','')

In [35]:
def write_output(py):
    code = py.replace("```py","").replace("```","")
    with open("optimized.py", "w") as f:
        f.write(code)

In [9]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [25]:
css = """
.python {background-color: #306998;}
.python-refactored {background-color: #050;}
"""

In [10]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far 

In [19]:
def select_sample_program(sample_program):
    if sample_program=="pi":
        return pi
    elif sample_program=="python_hard":
        return python_hard
    elif sample_program=="own_code":
        return ""
    else:
        return "Type your Python program here"

In [38]:
import gradio as gr

compiler_cmd = c_compiler_cmd("optimized")

with gr.Blocks(css=css) as ui:
    gr.Markdown("## Refactor python code")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        python_refactored = gr.Textbox(label="Refactored code:", lines=10)
    with gr.Row():
        with gr.Column():
            sample_program = gr.Radio(["python_hard", "pi", "own_code"], label="Sample program", value="python_hard")
            model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        python_refactored_run = gr.Button("Run refactored python")
    with gr.Row():
        python_out = gr.TextArea(label="Original python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="Refactored python result:", elem_classes=["python-refactored"])

    sample_program.change(select_sample_program, inputs=[sample_program], outputs=[python])
    convert.click(optimize, inputs=[python, model], outputs=[python_refactored])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    python_refactored_run.click(execute_python, inputs=[python_refactored], outputs=[cpp_out])

ui.launch()

* Running on local URL:  http://127.0.0.1:7929
* To create a public link, set `share=True` in `launch()`.
